In [1]:
! pip install --upgrade pip
! pip install -r requirements.txt

In [3]:
import os
import random
import pathlib
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

2025-03-12 14:48:45.920292: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-12 14:48:45.925815: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-12 14:48:45.938688: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741790925.953180   37987 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741790925.957627   37987 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1741790925.973550   37987 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [4]:
dataset_path = 'datasets/'
cat_path = os.path.join(dataset_path, 'Cat')
dog_path = os.path.join(dataset_path, 'Dog')

print(f'total cat images: {len(os.listdir(cat_path))}')
print(f'total of dog images: {len(os.listdir(dog_path))}')

total cat images: 12501
total of dog images: 12501


In [5]:
img_size = 128
data = []


for category in ['Cat', 'Dog']:
    path = os.path.join(dataset_path, category)
    label = 0 if category == 'Cat' else 1

    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)

        if not img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            print(f'Non-image file ignored: {img_name}')
            continue


        try:
            with Image.open(img_path) as img_test:
                img_test.verify()

            img = cv2.imread(img_path)
            if img is None:
                print(f'the image file is corrupted: {img_name}')
                continue


            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (img_size, img_size))
            data.append([img, label])


        except Exception as e:
            print(f'error with {img_name}: {e}')


random.shuffle(data)

print(f'Data processing is complete. Number of samples: {len(data)}')


the image file is corrupted: 11874.jpg
the image file is corrupted: 3300.jpg


Corrupt JPEG data: 99 extraneous bytes before marker 0xd9


the image file is corrupted: 10501.jpg
the image file is corrupted: 11210.jpg


Corrupt JPEG data: 239 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9


the image file is corrupted: 9778.jpg
the image file is corrupted: 660.jpg
the image file is corrupted: 2663.jpg
the image file is corrupted: 10820.jpg
the image file is corrupted: 9565.jpg
the image file is corrupted: 11935.jpg
the image file is corrupted: 8470.jpg
the image file is corrupted: 10404.jpg
error with 666.jpg: cannot identify image file '/workspaces/cats-vs-dogs-classifier/datasets/Cat/666.jpg'
the image file is corrupted: 5553.jpg
the image file is corrupted: 9171.jpg
the image file is corrupted: 140.jpg


...

the image file is corrupted: 5736.jpg
the image file is corrupted: 11853.jpg


Corrupt JPEG data: 226 extraneous bytes before marker 0xd9


the image file is corrupted: 7133.jpg
the image file is corrupted: 7112.jpg
the image file is corrupted: 10158.jpg
the image file is corrupted: 2877.jpg
the image file is corrupted: 10797.jpg
Non-image file ignored: Thumbs.db


Corrupt JPEG data: 399 extraneous bytes before marker 0xd9


the image file is corrupted: 3136.jpg
the image file is corrupted: 5604.jpg
the image file is corrupted: 10747.jpg
Data processing is complete. Number of samples: 24946


In [ ]:
X = np.zeros((len(data), img_size, img_size, 3), dtype=np.float32)
y = np.zeros(len(data), dtype=np.uint8)

for i, (img, label) in enumerate(data):
    X[i] = img / 255.0
    y[i] = label

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


2025-03-12 15:19:43.969851: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Accuracy on Test Data: {test_acc:.2f}')